In [1]:
import pandas as pd
import json
import numpy as np
import glob
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os
import plotly.io as pio
# The bin folder has the DLLs
os.environ['path'] += r';C:/Users/JKIM4/Downloads/vips-dev-w64-all-8.11.0/vips-dev-8.11/bin'
import pyvips

In [2]:
# setting directories
dir_code = "../run"
dir_data = dir_code + "/data"
dir_result = dir_code + "./results"

# setting configs json file path
file_configs = dir_code + "/configs.json"

# reading configs json file
print("reading configuration json file from = {}".format(file_configs))
with open(file_configs, "r") as read_file:
    configs = json.load(read_file)

configs['dir_code'] = dir_code
configs['dir_data'] = dir_data
configs['dir_result'] = dir_result
    
configs

reading configuration json file from = ../run/configs.json


{'dir_code': '../run',
 'dir_data': '../run/data',
 'dir_data_test': './data/test_data',
 'dir_data_stream': './data/stream_data',
 'dir_results': './results',
 'randomruns': 10,
 'weather': 'Norfolk',
 'labeling_methodology': 'Simple',
 'feature_selection_methodology': 'Embedded',
 'number_of_trees': 500,
 'simulation_timestep_min': 10,
 'fdd_reporting_frequency_hrs': 3,
 'split_test_size': 0.2,
 'train_test_apply': 'apply',
 'sensor_name_elec': 'electricity_facility [W]',
 'sensor_unit_elec': 'W',
 'sensor_name_ng': 'naturalgas_facility [W]',
 'sensor_unit_ng': 'W',
 'simulation_date_start': '2006 01-01',
 'simulation_date_end': '2007 01-01',
 'cost_est_timestep_min': 60,
 'dir_result': '../run./results'}

In [3]:
# reading FDD results file
df_result = pd.read_csv(configs["dir_result"] + "/{}_{}.csv".format(configs["weather"], configs["train_test_apply"]))
# df_result = pd.read_csv(configs["dir_result"] + "/{}_{}_ForTesting.csv".format(configs["weather"], configs["train_test_apply"]))
df_result

,output_teststream_data_preformatted_condenser_fouling_chiller_0.4.csvstream_data_preformatted_improperly_sized_equipment_boilers_1.5.csvstream_data_preformatted_improperly_sized_equipment_wshp_1.5.csvstream_data_preformatted_supply_air_duct_leakages_0.3.csvstream_data_preformatted_thermostat_bias_-3.0.csvstream_data_preformatted_thermostat_bias_3.0.csv
0,improperly_sized_equipment_chillers
1,improperly_sized_equipment_boilers
2,improperly_sized_equipment_boilers
3,improperly_sized_equipment_chillers
4,improperly_sized_equipment_chillers
...,...
17515,improperly_sized_equipment_boilers
17516,improperly_sized_equipment_boilers
17517,improperly_sized_equipment_boilers
17518,improperly_sized_equipment_boilers


In [4]:
# creating empty dataframe with timestamp
freq = str(configs['cost_est_timestep_min']) + 'min'
df_combined = pd.DataFrame([])
df_combined['reading_time'] = pd.date_range(configs['simulation_date_start'], configs['simulation_date_end'], freq=freq)
df_combined = df_combined.set_index(['reading_time'])[:-1]

df_combined

""
reading_time
2006-01-01 00:00:00
2006-01-01 01:00:00
2006-01-01 02:00:00
2006-01-01 03:00:00
2006-01-01 04:00:00
...
2006-12-31 19:00:00
2006-12-31 20:00:00
2006-12-31 21:00:00


In [5]:
# expanding FDD results into the same user-specified timestep
df_result_exp = np.repeat(list(df_result.values), configs['fdd_reporting_frequency_hrs']*int(60/configs['cost_est_timestep_min']))
df_result_exp = pd.DataFrame(df_result_exp)
df_result_exp.columns = ['FaultType']
df_result_exp.index = df_combined.index
df_result_exp

ValueError: Length mismatch: Expected axis has 52560 elements, new values have 8760 elements

In [ ]:
# setting timestamp for raw simulation data
freq_raw = str(configs['simulation_timestep_min']) + 'min'
df_index = pd.DataFrame([])
df_index['reading_time'] = pd.date_range(configs['simulation_date_start'], configs['simulation_date_end'], freq=freq_raw)
df_index = df_index.set_index(['reading_time'])[:-1]
timestamp_last = df_index.index[-1]

In [ ]:
# reading baseline simulation results
print("[Estimating Fault Cost] reading baseline simulation results")
df_baseline = pd.read_csv(configs['dir_data']+"/"+configs['weather']+"/baseline.csv", usecols=[configs['sensor_name_elec'], configs['sensor_name_ng']])
df_baseline.columns = [f"baseline_elec_{configs['sensor_unit_elec']}",f"baseline_ng_{configs['sensor_unit_ng']}"]
df_baseline.index = df_index.index
df_baseline = df_baseline.resample(str(configs['cost_est_timestep_min'])+"T").mean()
df_baseline

In [ ]:
# annual elec = 1356.2 GJ = 376722 kWh
# annual gas = 597.06 GJ = 165850 kWh
df_baseline.sum()/1000

In [ ]:
# # recreating FDD results with unique fault type (consecutive fault types are removed)
# df_unique = df_result_exp[(df_result_exp.ne(df_result_exp.shift())).any(axis=1)]
# df_unique = df_unique.reset_index()

# # reading individual fault simulation results (based on FDD results) and creating whole year combined results
# count = 1
# print("[Estimating Fault Cost] combining simulation results from the FDD results")

# df_combined_temp = pd.DataFrame()
# for index, row in df_unique.iterrows():
        
#     # specifying start and stop timestamp for each detected fault
#     rownum_current = df_unique.loc[df_unique.index==index,:].index[0]
#     timestamp_start = df_unique.iloc[rownum_current,:].reading_time
    
#     if rownum_current+1 < df_unique.shape[0]:
#         timestamp_end = df_unique.iloc[rownum_current+1,:].reading_time - pd.Timedelta(minutes=configs["simulation_timestep_min"])
#     else:
#         timestamp_end = timestamp_last
        
#     print(f"[Estimating Fault Cost] prossessing [{row['FaultType']} ({count}/{df_unique.shape[0]})] from the FDD results covering {timestamp_start} to {timestamp_end}")
        
#     count_file = 0
#     for file in glob.glob(configs['dir_data']+"/"+configs['weather']+f"/*{row['FaultType']}*"):
#         count_file += 1
#         print(f"[Estimating Fault Cost] count {count_file} | reading [{file}] file")
#         if count_file == 1:
#             df_temp = pd.read_csv(file, usecols=[configs['sensor_name_elec'], configs['sensor_name_ng']])
#             df_temp.index = df_index.index
#             df_temp = df_temp.resample(str(configs['cost_est_timestep_min'])+"T").mean()
#             df_temp = df_temp[timestamp_start:timestamp_end]
#             df_fault = df_temp.copy()
#         else:
#             df_temp = pd.read_csv(file, usecols=[configs['sensor_name_elec'], configs['sensor_name_ng']])
#             df_temp.index = df_index.index
#             df_temp = df_temp.resample(str(configs['cost_est_timestep_min'])+"T").mean()
#             df_temp = df_temp[timestamp_start:timestamp_end]
#             df_fault += df_temp
            
#     # averaging all fault intensity simulations for a single fault and merging into combined dataframe
#     print(f"[Estimating Fault Cost] averaging all fault intensity simulations for a single fault and merging into combined dataframe")
#     df_fault = df_fault/count_file
#     df_combined_temp = pd.concat([df_combined_temp, df_fault])
#     count+=1
    
# #     if count==3:
# #         break

In [ ]:
# df_combined_temp.to_csv("./df_combined_temp.csv")
df_combined_temp = pd.read_csv("./df_combined_temp.csv", index_col=0)
df_combined_temp.columns = [f"faulted_elec_{configs['sensor_unit_elec']}",f"faulted_ng_{configs['sensor_unit_ng']}"]
df_combined_temp.index = pd.to_datetime(df_combined_temp.index)
df_combined_temp

In [ ]:
# creating combined dataframe from baseline and faulted timeseries data
df_combined_temp.columns = [f"faulted_elec_{configs['sensor_unit_elec']}",f"faulted_ng_{configs['sensor_unit_ng']}"]
df_combined_temp.index = pd.to_datetime(df_combined_temp.index)
df_combined = pd.merge(df_combined, df_baseline, how='outer', left_index=True, right_index=True)
df_combined = pd.merge(df_combined, df_combined_temp, how='outer', left_index=True, right_index=True)

df_combined

In [ ]:
df_combined['diff_elec'] = df_combined["faulted_elec_{}".format(configs["sensor_unit_elec"])] - df_combined["baseline_elec_{}".format(configs["sensor_unit_elec"])]
df_combined['diff_ng'] = df_combined["faulted_ng_{}".format(configs["sensor_unit_ng"])] - df_combined["baseline_ng_{}".format(configs["sensor_unit_ng"])]

In [ ]:
# creating columns for time, date, and month
df_combined['Time'] = pd.to_datetime(df_combined.index).time
df_combined['Time'] = df_combined.Time.astype(str).str.rsplit(":",1, expand=True).iloc[:,0]
df_combined['Date'] = pd.to_datetime(df_combined.index).date
df_combined['Month'] = pd.to_datetime(df_combined.index).month

df_combined

In [ ]:
# calculate monthly and annual excess energy usages
if (configs['sensor_unit_ng']=='W') & (configs['sensor_unit_elec']=='W'):
    df_monthly = df_combined.groupby(['Month'])[["baseline_elec_{}".format(configs["sensor_unit_elec"]),"baseline_ng_{}".format(configs["sensor_unit_ng"]),'diff_elec','diff_ng']].sum()/1000/(60/configs['cost_est_timestep_min']) #convert W to kWh
    base_annual_elec = round(df_monthly["baseline_elec_{}".format(configs["sensor_unit_elec"])].sum()) # in kWh
    base_annual_ng = round(df_monthly["baseline_ng_{}".format(configs["sensor_unit_ng"])].sum()) # in kWh
    diff_annual_elec = round(df_monthly.sum()['diff_elec']) # in kWh
    diff_annual_ng = round(df_monthly.sum()['diff_ng']) # in kWh
    perc_annual_elec = round(diff_annual_elec/base_annual_elec*100, 3) # in %
    perc_annual_ng = round(diff_annual_ng/base_annual_ng*100, 3) # in %
else:
    # add other unit conversions
    print("[Estimating Fault Impact] unit conversion from {} for electricity and {} for natural gas to kWh is not currently supported".format(configs['sensor_unit_elec'],configs['sensor_unit_ng']))
    
df_monthly

In [ ]:
# plot setting
title_font_size = 12
colorbar_font_size = 12
tick_font_size = 12
anot_font_size = 14
fontfamily = 'verdana'
barwidth = 0.75
colorscale=[
    [0.0, 'rgb(5,48,97)'],
    [0.1, 'rgb(33,102,172)'],
    [0.2, 'rgb(67,147,195)'],
    [0.3, 'rgb(146,197,222)'],
    [0.4, 'rgb(209,229,240)'],
    [0.5, 'rgb(247,247,247)'],
    [0.6, 'rgb(253,219,199)'],
    [0.7, 'rgb(244,165,130)'],
    [0.8, 'rgb(214,96,77)'],
    [0.9, 'rgb(178,24,43)'],
    [1.0, 'rgb(103,0,31)']
]
# colorscale=[
#     [0.0, 'rgb(49,54,149)'],
#     [0.1, 'rgb(69,117,180)'],
#     [0.2, 'rgb(116,173,209)'],
#     [0.3, 'rgb(171,217,233)'],
#     [0.4, 'rgb(224,243,248)'],
#     [0.5, 'rgb(255,242,204)'],
#     [0.6, 'rgb(254,224,144)'],
#     [0.7, 'rgb(253,174,97)'],
#     [0.8, 'rgb(244,109,67)'],
#     [0.9, 'rgb(215,48,39)'],
#     [1.0, 'rgb(165,0,38)']
# ]
color_bar = 'rgb(116,173,209)'
range_max_elec = max( df_combined['diff_elec'].max() , abs(df_combined['diff_elec'].min()) )
range_max_ng = max( df_combined['diff_ng'].max() , abs(df_combined['diff_ng'].min()) )


# plotting
num_rows = 2
num_cols = 3
fig = make_subplots(
    rows=num_rows, 
    cols=num_cols, 
    shared_xaxes=True, 
    vertical_spacing=0.025,
    horizontal_spacing=0.1,
    column_widths=[0.2, 0.4, 0.4],
)  

# heatmap
fig.add_trace(go.Heatmap(
    z=df_combined['diff_elec'],
    x=df_combined['Date'],
    y=df_combined['Time'],
    colorscale='tempo',
    coloraxis='coloraxis1',
),
row=1, col=3)
fig.add_trace(go.Heatmap(
    z=df_combined['diff_ng'],
    x=df_combined['Date'],
    y=df_combined['Time'],
    colorscale='tempo',
    coloraxis='coloraxis2',
),
row=2, col=3)

# bar chart
fig.add_trace(go.Bar(
    x=df_monthly.index,
    y=df_monthly['diff_elec'],
    showlegend=False,
    width=barwidth,
    marker=dict(
        color=color_bar,
    )
),
row=1, col=2)
fig.add_trace(go.Bar(
    x=df_monthly.index,
    y=df_monthly['diff_ng'],
    showlegend=False,
    width=barwidth,
    marker=dict(
        color=color_bar,
    )
),
row=2, col=2)

# annotation
if perc_annual_elec > 0:
    text_elec = "+"
else:
    text_elec = ""

if perc_annual_ng > 0:
    text_ng = "+"
else:
    text_ng = ""
fig.add_annotation(
    x=0.08,
    y=0.75,
    xref="paper",
    yref="paper",
    xanchor='center',
    yanchor='middle',
    text="Excess<br>electricity<br><b>{} kWh/year<br>({}{}%)</b>".format(diff_annual_elec, text_elec, perc_annual_elec),
    font=dict(
        family=fontfamily,
        size=anot_font_size,
        ),
    showarrow=False,
    align="right",
    )
fig.add_annotation(
    x=0.08,
    y=0.25,
    xref="paper",
    yref="paper",
    xanchor='center',
    yanchor='middle',
    text="Excess<br>natural gas<br><b>{} kWh/year<br>({}{}%)</b>".format(diff_annual_ng, text_ng, perc_annual_ng),
    font=dict(
        family=fontfamily,
        size=anot_font_size,
        ),
    showarrow=False,
    align="right",
    )
fig.add_annotation(
    x=0.2,
    y=0.75,
    xref="paper",
    yref="paper",
    xanchor='center',
    yanchor='middle',
    text="<b>Electricity [kWh]</b>",
    textangle=270,
    font=dict(
        family=fontfamily,
        size=title_font_size,
        ),
    showarrow=False,
    align="center",
    )
fig.add_annotation(
    x=0.2,
    y=0.25,
    xref="paper",
    yref="paper",
    xanchor='center',
    yanchor='middle',
    text="<b>Natural gas [kWh]</b>",
    textangle=270,
    font=dict(
        family=fontfamily,
        size=title_font_size,
        ),
    showarrow=False,
    align="center",
    )

# layout
fig.update_layout(
    width=900,
    height=400,
    margin=dict(
        l=0,
        r=0,
        t=0,
        b=0,
    ),
    plot_bgcolor='white',
    coloraxis1=dict(
        cmin=-range_max_elec,
        cmid=0,
        cmax=range_max_elec,
        colorscale=colorscale, 
        colorbar = dict(
            title=dict(
                text = "Excess electricty [{}]".format(configs['sensor_unit_elec']),
                side='right',
                font=dict(
                    size=colorbar_font_size,
                    family=fontfamily,
                ),
            ),
            len=0.5,
            x=1,
            xanchor='left',
            y=0.75,
            yanchor='middle',
            thickness=23,
        )
    ),
    coloraxis2=dict(
        cmin=-range_max_ng,
        cmid=0,
        cmax=range_max_ng,
        colorscale=colorscale, 
        colorbar = dict(
            title=dict(
                text = "Excess natural gas [{}]".format(configs['sensor_unit_ng']),
                side='right',
                font=dict(
                    size=colorbar_font_size,
                    family=fontfamily,
                ),
            ),
            len=0.5,
            x=1,
            xanchor='left',
            y=0.25,
            yanchor='middle',
            thickness=23,
        ),
    ),
)

# axes
for row in range(1, num_rows+1):
    for col in range(1, num_cols+1):
        if col==1:
            fig.update_yaxes(
                showticklabels=False,
                row=row, col=col
            )      
        elif col==2:     
            fig.update_yaxes(
                tickfont = dict(
                    family=fontfamily,
                    size=tick_font_size,
                ),
                row=row, col=col
            )
        elif col==3:
            fig.update_yaxes(
                title = dict( 
                    text="<b>Time</b>",
                    font=dict(
                        family=fontfamily,
                        size=title_font_size,
                    ),
                    standoff=0,
                ),
                tickfont = dict(
                    family=fontfamily,
                    size=tick_font_size,
                ),
                row=row, col=col
            )      

fig.update_xaxes(
    title = dict( 
        text="<b>Date</b>",
        font=dict(
            family=fontfamily,
            size=title_font_size,
        ),
    ),
    tickfont = dict(
        family=fontfamily,
        size=tick_font_size,
    ),
    tickformat="%b",
    dtick="M2",
    row=2, col=3
)
fig.update_xaxes(
    title = dict( 
        text="<b>Month</b>",
        font=dict(
            family=fontfamily,
            size=title_font_size,
        ),
    ),
    tickfont = dict(
        family=fontfamily,
        size=tick_font_size,
    ),
    row=2, col=2
)
fig.update_xaxes(
    dtick=1,
    row=1, col=2
)


# pio.write_image(fig, "./figures/fig_comparison_heatmap.svg")
# # https://stackoverflow.com/questions/51450134/how-to-convert-svg-to-png-or-jpeg-in-python
# image = pyvips.Image.thumbnail("./figures/fig_comparison_heatmap.svg", 3000)
# image.write_to_file("./figures/fig_comparison_heatmap.png")

fig.show()